# Accounting demo notebook.
This notebook demonstrates the operation of an accounting application for processing travel reports.

We upload a pre-formed workflow to the server, which can be created and edited using the UI. Then, business entities such as 'employee', 'expense report', and 'payment' are generated in the notebook to register entity models. We will save some of the entities (employees, expense reports) to the database, and the other part (payments) will be created and saved by the external calculation node, which needs to be running in parallel with this notebook.

After saving the entities, we initiate changes (launch transitions), simulating the following business process. Employees create an expense report after a business trip. Then, various life cycle scenarios for this entity are randomly worked out. They update/delete/restore/send it for review to accounting department and manager. They, in turn, approve/reject/apply calculations/initiate payment, etc. The external calculation node listens to the server and, if necessary, executes events triggered by the workflow and provided in the Calculation Processor (create, update entities, launch transition, etc.).

In [1]:
# BEFORE START: kindly create a file 'application-env.properties' in the folder of this notebook and provide the credentials. Example:
#cyoda.host=http://localhost:8082/api
#cyoda.name=name
#cyoda.password=password

!pip install dicttoxml
!pip install faker

In [ ]:
# HTTP Client

import requests

class HttpClient:
    def __init__(self, file_path='application-env.properties'):
        self.session = requests.Session()
        self.session.headers.update({
            "Content-Type": "application/json",
            "X-Requested-With": "XMLHttpRequest"
        })

        creds = self.read_credentials(file_path)
        self.base_url = creds.get('cyoda.host')
        self.username = creds.get('cyoda.name')
        self.password = creds.get('cyoda.password')

        self.authenticate()

    @staticmethod
    def read_credentials(file_path):
        credentials = {}
        try:
            with open(file_path, 'r') as file:
                for line in file:
                    key, value = line.strip().split('=')
                    credentials[key] = value
        except FileNotFoundError:
            raise Exception(f"File {file_path} not found")
        except ValueError:
            raise Exception(f"Error parsing credentials in {file_path}")
        return credentials

    def authenticate(self):
        if not self.base_url or not self.username or not self.password:
            raise Exception("Base URL, username, or password is missing")

        url = f"{self.base_url}/auth/login"
        response = self.session.post(url, json={"username": self.username, "password": self.password})
        response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
        token = response.json().get('token')
        if token:
            self.session.headers.update({"Authorization": f"Bearer {token}"})
        else:
            raise Exception("Authentication failed, token not found")

    def request(self, method, endpoint, **kwargs):
        url = f"{self.base_url}{endpoint}"

        if 'json' in kwargs:
            self.session.headers.update({"Content-Type": "application/json"})
        elif 'data' in kwargs:
            data = kwargs['data']
            if isinstance(data, str) and data.strip().startswith('<'):
                self.session.headers.update({"Content-Type": "application/xml"})
            else:
                self.session.headers.update({"Content-Type": "application/x-www-form-urlencoded"})
        else:
            self.session.headers.pop("Content-Type", None)

        try:
            response = self.session.request(method, url, **kwargs)
            response.raise_for_status()

            if response.content.strip():
                try:
                    return response.json()
                except ValueError:
                    print("Response is not JSON, returning as text.")
                    return response.text
            else:
                print(f"Request was successful.")
                return None
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")
            print(f"Response content: {response.text}")
        except ValueError as json_err:
            print(f"JSON parsing error occurred: {json_err}")
            print(f"Response content: {response.text}")
            return None
        except Exception as err:
            print(f"Other error occurred: {err}")

client = HttpClient()
print("Client initialized and authenticated.")

In [ ]:
# Entity Generator
from faker import Faker
import json
import uuid
from datetime import datetime, timedelta
from decimal import Decimal
import random

class EntityGenerator:
    def __init__(self):
        self.faker = Faker()
        self.dummy_uuid = uuid.uuid1()
        self.descriptions = ["hotel", "taxi", "transportation", "meals", "other"]
        self.departments = ["Accounting", "Marketing", "HR", "IT", "Sales"]
        self.cities = ["Amsterdam", "Barcelona", "Berlin", "Vienna", "Copenhagen",
                       "Lisbon", "Madrid", "Milan", "Paris", "Prague"]

    def generate_expense_reports(self, count, employee_ids=None):
        reports = []
        id_list = employee_ids if employee_ids else [self.dummy_uuid]

        for _ in range(count):
            employee_id = random.choice(id_list)

            report = {
                "employeeId": str(employee_id),
                "destination": random.choice(self.cities),
                "departureDate": (datetime.now() - timedelta(days=1)).isoformat() + 'Z',
                "expenseList": self.generate_expense_list(2),
                "advancePayment": float(Decimal(self.faker.pydecimal(left_digits=2, right_digits=2, positive=True))),
                "amountPayable": 0.00
            }
            reports.append(report)

        return json.dumps(reports, indent=2)

    def generate_employees(self, count):
        employees = []
        for _ in range(count):
            employee = {
                "fullName": self.faker.name(),
                "department": random.choice(self.departments)
            }
            employees.append(employee)

        return json.dumps(employees, indent=2)

    def generate_payments(self, count):
        payments = []
        for _ in range(count):
            payment = {
                "expenseReportId": str(self.dummy_uuid),
                "amount": float(Decimal(self.faker.pydecimal(left_digits=4, right_digits=2, positive=True)))
            }
            payments.append(payment)

        return json.dumps(payments, indent=2)
    
    def generate_expense_list(self, count):
        expenses = []
        for _ in range(count):
            expense = {
                "description": random.choice(self.descriptions),
                "amount": float(Decimal(self.faker.pydecimal(left_digits=2, right_digits=2, positive=True)))
            }
            expenses.append(expense)

        return expenses

generator = EntityGenerator()

In [ ]:
# Entity Service
from functools import wraps
from pprint import pprint

def print_response_decorator(func):
    @wraps(func)
    def wrapper(self, *args, print_response=False, **kwargs):
        response = func(self, *args, **kwargs)

        if print_response:
            method_name = func.__name__
            print(f"\nResponse({method_name}):")
            pprint(response)

        return response

    return wrapper

class EntityService:
    def __init__(self, client):
        self.client = client
        self.entityClassName = "com.cyoda.tdb.model.treenode.TreeNodeEntity"

    # entity model endpoints
    @print_response_decorator
    def save_model(self, entity_spec, payload, print_response=False):
        format = entity_spec.get("format")
        converter = entity_spec.get("converter")
        entityName = entity_spec.get("entityName")
        modelVersion = entity_spec.get("modelVersion")
        endpoint = f"/treeNode/model/import/{format}/{converter}/{entityName}/{modelVersion}"
        response = self.client.request("POST", endpoint, json=payload)
        return response

    @print_response_decorator
    def get_model(self, entity_spec):
        converter = "JSON_SCHEMA" # "SIMPLE_VIEW"
        entityName = entity_spec.get("entityName")
        modelVersion = entity_spec.get("modelVersion")
        endpoint = f"/treeNode/model/export/{converter}/{entityName}/{modelVersion}"
        response = self.client.request("GET", endpoint)
        return response

    @print_response_decorator
    def lock_model(self, entity_spec):
        entityName = entity_spec.get("entityName")
        modelVersion = entity_spec.get("modelVersion")
        endpoint = f"/treeNode/model/{entityName}/{modelVersion}/lock"
        response = self.client.request("PUT", endpoint)
        return response

    @print_response_decorator
    def get_all_models(self):
        endpoint = f"/treeNode/model/"
        response = self.client.request("GET", endpoint)
        return response

    @print_response_decorator
    def delete_model(self, entity_spec):
        entityName = entity_spec.get("entityName")
        modelVersion = entity_spec.get("modelVersion")
        endpoint = f"/treeNode/model/{entityName}/{modelVersion}"
        response = self.client.request("DELETE", endpoint)
        return response

    # entity endpoints
    @print_response_decorator
    def save_entities(self, entity_spec, payload):
        format = entity_spec.get("format")
        entityType = entity_spec.get("entityType")
        entityName = entity_spec.get("entityName")
        modelVersion = entity_spec.get("modelVersion")
        endpoint = f"/entity/{format}/{entityType}/{entityName}/{modelVersion}"
        response = self.client.request("POST", endpoint, json=payload)
        return response

    @print_response_decorator
    def get_entity(self, entity_spec, entityId):
        entityType = entity_spec.get("entityType")
        endpoint = f"/entity/{entityType}/{entityId}"
        response = self.client.request("GET", endpoint)
        return response

    @print_response_decorator
    def get_all_entities(self, entity_spec):
        entityType = entity_spec.get("entityType")
        entityName = entity_spec.get("entityName")
        modelVersion = entity_spec.get("modelVersion")
        endpoint = f"/entity/{entityType}/{entityName}/{modelVersion}"
        response = self.client.request("GET", endpoint)
        return response

    @print_response_decorator
    def update_entity(self, entity_spec, entityId, transition, payload):
        format = entity_spec.get("format")
        entityType = entity_spec.get("entityType")
        endpoint = f"/entity/{format}/{entityType}/{entityId}/{transition}"
        response = self.client.request("PUT", endpoint, json=payload)
        return response

    @print_response_decorator
    def delete_single_entity(self, entity_spec, entityId):
        entityType = entity_spec.get("entityType")
        endpoint = f"/entity/{entityType}/{entityId}"
        response = self.client.request("DELETE", endpoint)
        return response

    @print_response_decorator
    def delete_entities(self, entity_spec):
        entityType = entity_spec.get("entityType")
        entityName = entity_spec.get("entityName")
        modelVersion = entity_spec.get("modelVersion")
        endpoint = f"/entity/{entityType}/{entityName}/{modelVersion}"
        response = self.client.request("DELETE", endpoint)
        return response

    # auxiliary endpoints
    @print_response_decorator
    def get_current_state(self, entityId):
        endpoint = f"/platform-api/entity-info/fetch/lazy?entityClass={self.entityClassName}&entityId={entityId}&columnPath=state"
        response = self.client.request("GET", endpoint)
        return response[0]['value']

    @print_response_decorator
    def get_available_transitions(self, entityId):
        endpoint = f"/platform-api/entity/fetch/transitions?entityId={entityId}&entityClass={self.entityClassName}"
        response = self.client.request("GET", endpoint)
        return response

    @print_response_decorator
    def launch_transition(self, entityId, transitionName):
        endpoint = f"/platform-api/entity/transition?entityId={entityId}&entityClass={self.entityClassName}&transitionName={transitionName}"
        response = self.client.request("PUT", endpoint)
        return response

    @print_response_decorator
    def load_workflow(self, file_name):
        endpoint = f"/platform-api/statemachine/import?needRewrite=true"
        file_path = file_name

        with open(file_path, 'r', encoding='utf-8') as file:
            json_payload = json.load(file)

        response = self.client.request("POST", endpoint, json=json_payload)
        return response

service = EntityService(client)

In [ ]:
# Entity Model specification
employee_spec = {
    "entityName": "employee",
    "modelVersion": "1",
    "converter": "SAMPLE_DATA",
    "entityType": "TREE",
    "format": "JSON"
}

expense_report_spec = {
    "entityName": "expense_report",
    "modelVersion": "1",
    "converter": "SAMPLE_DATA",
    "entityType": "TREE",
    "format": "JSON"
}

payment_spec = {
    "entityName": "payment",
    "modelVersion": "1",
    "converter": "SAMPLE_DATA",
    "entityType": "TREE",
    "format": "JSON"
}

In [ ]:
# Upload workflow
service.load_workflow('workflow_for_Payment_Expense Report.json')

In [ ]:
# Delete entities
# employee
service.delete_entities(employee_spec)

# expense_report
service.delete_entities(expense_report_spec)

# payment
service.delete_entities(payment_spec)

In [ ]:
# Delete models
# employee
service.delete_model(employee_spec)

# expense_report
service.delete_model(expense_report_spec)

# payment
service.delete_model(payment_spec)

In [ ]:
# Generate, save and lock models
# employee
employees = generator.generate_employees(1)

service.save_model(employee_spec, payload=json.loads(employees), print_response=True)
service.lock_model(employee_spec)

# expense_report
expense_reports = generator.generate_expense_reports(1)

service.save_model(expense_report_spec, payload=json.loads(expense_reports), print_response=True)
service.lock_model(expense_report_spec)

# payment
payments = generator.generate_payments(1)

service.save_model(payment_spec, payload=json.loads(payments), print_response=True)
service.lock_model(payment_spec)

In [ ]:
# Get all models
service.get_all_models()

In [ ]:
# View model
pprint(service.get_model(employee_spec))

pprint(service.get_model(expense_report_spec))

pprint(service.get_model(payment_spec))

In [ ]:
# Generate and save employees and their expense reports
employees_num = 3
reports_num = 10

employees = generator.generate_employees(employees_num)
response_employees = service.save_entities(employee_spec, payload=json.loads(employees), print_response=True)
employee_id_list = response_employees[0]['entityIds']

expense_reports = generator.generate_expense_reports(reports_num, employee_id_list)
response_expense_reports = service.save_entities(expense_report_spec, payload=json.loads(expense_reports), print_response=True)
expense_reports_id_list = response_expense_reports[0]['entityIds']

In [ ]:
# Get all employees
response_employees = service.get_all_entities(employee_spec, print_response=True)

In [ ]:
# Get all expense_reports
response_expense_reports = service.get_all_entities(expense_report_spec, print_response=True)

In [ ]:
# Get IDs of expense reports
expense_reports_id_list = [item['id'] for item in response_expense_reports]

pprint(expense_reports_id_list)

In [ ]:
import time

# It is recommended to view logs of the 'accounting demo' java application (the external calculation node) while this cell is running.

# Here we set a number of transitions to be applied to randomly chosen expense reports. Some transitions can be excluded for demo purpose (to skip 'delete-restore' and 'submit-reject' cycles). Some (like 'POST_PAYMENT') are automated and not designed to be pushed manually.

#excluded_transitions = {"DELETE", "REJECT_BY_ACCOUNTING", "REJECT_BY_MANAGER", "POST_PAYMENT"}
excluded_transitions = {"POST_PAYMENT"}

counter = 1
transition_num = 30 # Set any number

while counter <= transition_num:
    random_report_id = random.choice(expense_reports_id_list)
    print("\n#{} Expense Report ID: {}".format(counter, random_report_id))
    transitions = service.get_available_transitions(random_report_id)
    if transitions:
        random_transition = random.choice(transitions)

        if random_transition not in excluded_transitions:
            print("Transition chosen:\n'{}'".format(random_transition))
            service.launch_transition(random_report_id, random_transition)
        else:
            print("Transition is excluded from launching:\n'{}'".format(random_transition))        
    
    else:
        print("No transition available.")

    time.sleep(0.5) # Add a small delay between requests for ease of viewing logs
    counter += 1

In [ ]:
# Get all payments. The payment is created automatically when the expense report passes all checks and the travel expenses exceed the advance payment.
response_payments = service.get_all_entities(payment_spec, print_response=True)